In [1]:
KEYWORDS = [
    "traders","trading", "enterprise", "garments", "collection", "food", "clothes",'fashion', 
    "glass", "fittings", "digital", "kirana", "medical", "agency","tex",
    "security", "systems", "badges", "hospitality", "jewellers", 
    "ready-made", "store", "hospital", "restaurant", "auto", "center", 
    "dairy", "home", "products", "services", "furniture", "hardware", 
    "pharmacy", "stationery", "treatments", "nutrition", "wellness", 
    "sweets", "resort", "kitchen", "clothing", "fashion", "market", 
    "poultry", "seeds", "pesticides", "sales", "cafe", "clinic", 
    "supermart", "distributors", "automobiles", "electricity", 
    "electronics", "general", "provision", "fertilizers", "agriculture", 
    "beverages", "textiles", "plumbing", "supplies", "handicrafts", 
    "construction", "medical", "bakery", "tissue", "cleaning", 
    "appliances", "homecare", "kitchenware", "decor", "glass and fittings",
    "interiors", "shopping", "crafts", "tools", "wholesale", 
    "retail", "outlet", "merchants", "trade", "distribution", 
    "solutions", "innovation", "consultancy", "services", "equipment", 
    "manufacturing", "exports", "imports", "packaging", "network", 
    "consultants", "transport", "moving", "storage", "logistics", 
    "construction", "real estate", "brokerage", "management", 
    "finance", "investment", "funding", "support", "technology", 
    "software", "applications", "digital marketing", "advertising", 
    "communication", "entertainment", "events", "tourism", "travel", 
    "transportation", "automotive", "services", "supply chain", 
    "fashion", "cosmetics", "beauty", "spa", "wellness", "glass and fitting",
    "personal care", "gifts", "custom", "specialty", "craftsmanship", 
    "fashions", "motors", "enterprises", "garment", "cloth centre", "mart", 
    "foods", "silk and readymade", "wool centre", "jewellery", "mill", 
    "farms", "farm", "electrical", "egg centre", "centre", 
    "vegetable and fruits", "vegetables", "fruits", "pvt", "pvt ltd", 
    "limited", "solutions", "energies", "photo", "studio", "works", 
    "associates", "medico", "agencies", "diagnosis", "cool drinks", 
    "drinks", "care", "liquor", "automobiles", "materials", "diagnostics", 
    "provision", "trader", "farms", "farm", "stations", "restaurant", 
    "creations", "travels", "hardware", "printers", "graphics", 
    "fertilisers", "house", "studio", "private", "appliances", "steels", 
    "shop", "metals", "international", "jwellers", "corporation", 
    "dresses", "industries", "electricals", "company", "lim", "colddrinks", 
    "electron", "medicines", "llc", "computers", "hotel", "spa", 
    "cosmetics", "telecom", "sarees", "petroleums", "bhandar",'store','stores', 
    "surgical", "wines", "constructions", "shoppy", "lab", "builders", 
    "footwear", "wear", "shoe", "repair", "ventures", "paint", "depot",'cake','chinies',
    "tent", "decorators", "communications", "pharmacy", "products",'textile','CERAMIC','Pharmaceuticals',
]

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from fuzzywuzzy import fuzz
import re
import numpy as np

threshold = 0.80


##------------------------------------Fuzzy Wuzzy Layer-------------------------------------------------------------------------------------------------------


## Layer 1 - Fuzzy Wuzzy Without Preprocessing
def calculate_fuzzy_similarity(name1, name2):
    fuzzy_ratio = fuzz.ratio(name1, name2) / 100.0
    fuzzy_partial_ratio = fuzz.partial_ratio(name1, name2) / 100.0
    fuzzy_token_sort_ratio = fuzz.token_sort_ratio(name1, name2) / 100.0
    fuzzy_token_set_ratio = fuzz.token_set_ratio(name1, name2) / 100.0

    fuzzy_similarity = (fuzzy_ratio + fuzzy_partial_ratio + fuzzy_token_sort_ratio + fuzzy_token_set_ratio) / 4.0
    return fuzzy_similarity

#-----------------------------------------Data Preprocessing anf Framework--------------------------------------------------------------------------------------------------


# layer 4 over here 
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance as levenshtein_distance
import jellyfish

# Regular expressions for different patterns
SPECIAL_CHAR_DOT_REGEX = r"[.]"
SPECIAL_CHARS_REGEX = r"[-+.^:,_/\s]+" 
SALUTATION_REGEX = r"^(shree|shri|miss|smt|mrs|mr|ms|dr|master|hon|sir|madam|prof|capt|major|rev|fr|br)\s*"
PARENT_SPOUSE_NAME_REGEX = r"(?:\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of)\s*)"
COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX = r"\b(mohammad|mohammed|muhamed|mohd|mohamed|mohamad|muhamad|muhammad|muhammed|muhammet|mohamud|mohummad|mohummed|mouhamed|muhamaad|mohammod|mouhamad|mo|md|mahmood|mahmud|ahmad|ahmed|hameed|hamid|hammed|mahd|mahmod|mohd|mouhammed|mohamad|muhmood|mohhammed|muhmamed|mohmed|mohmat|muhmat|mu|m|shaikh|mo)\b"
LAST_NAMES_AGARWAL_VARIANTS_REGEX = r"\b(aggarwal|agrawal|agarwal|aggrawal|agarwalla|agarwal)\b"


# Preprocessing functions
def convert_to_lower(name):
    return name.lower()

def replace_adjacent_duplicates(value):
    if isinstance(value, str):
        return re.sub(r'(.)\1+', r'\1', value)
    return value

def replace_characters(name):
    replacements = {'e': 'i', 'j': 'z', 'v': 'w', 'q': 'k'}
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name

def replace_bigrams(name):
    replacements = {'ph': 'f', 'gh': 'g', 'th': 't', 'kh': 'k', 'dh': 'd', 'ch': 'c', 'sh': 's', 'au': 'o',
                    'bh': 'b', 'ks': 'x', 'ck': 'k', 'ah': 'h', 'wh': 'w', 'wr': 'r'}
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name

def remove_extra_spaces(name):
    return re.sub(r'\s+', ' ', name).strip()

def remove_consonant_a(name):
    consonants = 'bcdfghjklmnpqrstvwxyz'
    new_name = ''.join([name[i] for i in range(len(name)) if not (i > 0 and name[i] == 'a' and name[i - 1].lower() in consonants)])
    return new_name

def remove_special_characters(text):
    text = re.sub(SPECIAL_CHAR_DOT_REGEX, '', text)
    text = re.sub(SPECIAL_CHARS_REGEX, '', text)
    return text.strip()

def remove_salutations(text):
    return re.sub(SALUTATION_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_parent_spouse_name(text):
    return re.sub(r'\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of|daughter|son|child of)\s*[\w\s,.]*$', '', text, flags=re.IGNORECASE).strip()

def remove_common_muslim_variations(text):
    return re.sub(COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_agarwal_variants(text):
    return re.sub(LAST_NAMES_AGARWAL_VARIANTS_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_stop_words(text):
    stop_words = ['devi', 'dei', 'debi', 'kumar', 'kumaar', 'kumari', 'kumaari', 'kmr', 'kumr', 'bhai', 'bhau', 'bai', 'ben', 'singh', 'kaur', 'Md', 'Mohd', 'Mohammad', 'Mohamad']
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def preprocess(name):
    name = remove_salutations(name)
    name = remove_parent_spouse_name(name)
    name = remove_common_muslim_variations(name)
    name = remove_agarwal_variants(name)
    name = convert_to_lower(name)
    name = replace_adjacent_duplicates(name)
    name = replace_characters(name)
    name = replace_bigrams(name)
    name = remove_consonant_a(name)
    name = remove_special_characters(name)
    name = remove_extra_spaces(name)
    name = remove_stop_words(name)
    return name

# Load pre-trained models
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

# Similarity functions
def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2).item()

def calculate_levenshtein_similarity(name1, name2):
    lev_distance = levenshtein_distance(name1, name2)
    max_len = max(len(name1), len(name2))
    return (max_len - lev_distance) / max_len if max_len > 0 else 1.0

def calculate_phonetic_similarity(name1, name2):
    soundex1 = jellyfish.soundex(name1)
    soundex2 = jellyfish.soundex(name2)
    return jellyfish.jaro_winkler_similarity(soundex1, soundex2)

def calculate_jaccard_similarity(name1, name2):
    set1, set2 = set(name1), set(name2)
    intersection, union = set1.intersection(set2), set1.union(set2)
    return len(intersection) / len(union) if union else 1.0

##------------------------------------Calling Name_Match------------------------------------------------------------------------------------------------------

# Layer 2 similarity function
def name_match(name1, name2):
    name1_processed = preprocess(name1)
    name2_processed = preprocess(name2)

    embedding1 = get_embedding(name1_processed)
    embedding2 = get_embedding(name2_processed)
    embedding_similarity = calculate_cosine_similarity(embedding1, embedding2)

    levenshtein_similarity = calculate_levenshtein_similarity(name1_processed, name2_processed)
    phonetic_similarity = calculate_phonetic_similarity(name1_processed, name2_processed)
    jaccard_similarity = calculate_jaccard_similarity(name1_processed, name2_processed)

    fuzzy_ratio = fuzz.ratio(name1_processed, name2_processed) / 100.0
    fuzzy_partial_ratio = fuzz.partial_ratio(name1_processed, name2_processed) / 100.0
    fuzzy_token_sort_ratio = fuzz.token_sort_ratio(name1_processed, name2_processed) / 100.0
    fuzzy_token_set_ratio = fuzz.token_set_ratio(name1_processed, name2_processed) / 100.0

    fuzzy_similarity = (fuzzy_ratio + fuzzy_partial_ratio + fuzzy_token_sort_ratio + fuzzy_token_set_ratio) / 4.0

    # Weighted average of similarity metrics
    final_score = (
        embedding_similarity * 0.3 +
        levenshtein_similarity * 0.2 +
        phonetic_similarity * 0.3 +
        jaccard_similarity * 0.4 +
        fuzzy_similarity * 0.6
    )
    return final_score

#----------------------------------------Initials Matching Without Preprocessing---------------------------------------------------------------------------------------------------


# Layer 3 - Initials Matching Without Preprocessing
def check_initial_similarity(name1, name2):
    initials1 = [word[0].upper() for word in name1.split() if word]
    initials2 = [word[0].upper() for word in name2.split() if word]

    if not initials1 or not initials2:
        return 0.0
    
    return 1.0 if set(initials1).issubset(set(initials2)) or set(initials2).issubset(set(initials1)) else 0.0




##------------------------------------Fuzzy With Data Preprocessing-------------------------------------------------------------------------------------------------------

SPECIAL_CHAR_DOT_REGEX = r"[.]"
SPECIAL_CHARS_REGEX = r"[-+.^:,_/\s]+" 
SALUTATION_REGEX = r"^(shree|shri|miss|smt|mrs|mr|ms|dr|master|hon|sir|madam|prof|capt|major|rev|fr|br)\s*"
PARENT_SPOUSE_NAME_REGEX = r"(?:\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of)\s*)"
COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX = r"\b(mohammad|mohammed|muhamed|mohd|mohamed|mohamad|muhamad|muhammad|muhammed|muhammet|mohamud|mohummad|mohummed|mouhamed|muhamaad|mohammod|mouhamad|mo|md|mahmood|mahmud|ahmad|ahmed|hameed|hamid|hammed|mahd|mahmod|mohd|mouhammed|mohamad|muhmood|mohhammed|muhmamed|mohmed|mohmat|muhmat|mu|m|shaikh|mo)\b"
LAST_NAMES_AGARWAL_VARIANTS_REGEX = r"\b(aggarwal|agrawal|agarwal|aggrawal|agarwalla|agarwal)\b"


# Preprocessing functions
def convert_to_lower(name):
    return name.lower()

def remove_extra_spaces(name):
    return re.sub(r'\s+', ' ', name).strip()

def remove_common_muslim_variations(text):
    return re.sub(COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_agarwal_variants(text):
    return re.sub(LAST_NAMES_AGARWAL_VARIANTS_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_stop_words(text):
    stop_words = ['devi', 'dei', 'debi', 'kumar', 'kumaar', 'kumari', 'kumaari', 'kmr', 'kumr', 'bhai', 'bhau', 'bai', 'ben', 'singh', 'kaur', 'Md', 'Mohd', 'Mohammad', 'Mohamad']
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


 #  processing only for failed  of layer 3
def preprocess_FuzzyWuzzy(name):
    """Process a name through all the defined normalization steps."""
    name = convert_to_lower(name)
    name = remove_stop_words(name)
    name = remove_common_muslim_variations(name)
    name = remove_agarwal_variants(name)
    return name

def calculate_fuzzy_similarity_processed(name1, name2):
    name1 = preprocess_FuzzyWuzzy(name1)
    name2 = preprocess_FuzzyWuzzy(name2)
    
    fuzzy_ratio = fuzz.ratio(name1, name2) / 100.0
    fuzzy_partial_ratio = fuzz.partial_ratio(name1, name2) / 100.0
    fuzzy_token_sort_ratio = fuzz.token_sort_ratio(name1, name2) / 100.0
    fuzzy_token_set_ratio = fuzz.token_set_ratio(name1, name2) / 100.0

    fuzzy_similarity = (fuzzy_ratio + fuzzy_partial_ratio + fuzzy_token_sort_ratio + fuzzy_token_set_ratio) / 4.0
    return fuzzy_similarity

def check_keywords_in_names(name1, name2):
    """Check for presence of keywords in either or both names."""
    found_in_name1 = any(keyword in name1.lower() for keyword in KEYWORDS)
    found_in_name2 = any(keyword in name2.lower() for keyword in KEYWORDS)
    
    if found_in_name1 and found_in_name2:
        return 1  # Flag remains 1 if keyword found in both names
    elif found_in_name1 or found_in_name2:
        return 0  # Flag set to 0 if keyword found in either name
    return 1



/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    classification_report
)

def process_name_matching(file_path):
# def process_name_matching(file_path, num_rows=10):

    # Load data
    df = pd.read_csv(file_path)
    # df=df.head(100)
    # df = pd.read_csv(file_path, nrows=num_rows)
    threshold = 0.80

    # Layer 1: Fuzzy Matching Without Preprocessing
    df["First_Layer_Score"] = df.apply(lambda x: calculate_fuzzy_similarity(x['name1'], x['name2']), axis=1)
    df["First_Layer_Pass"] = df["First_Layer_Score"] >= threshold
    df_layer2 = df[~df["First_Layer_Pass"]].copy()

    # Layer 2: Preprocessed Fuzzy Matching
    df_layer2["Second_Layer_Score"] = df_layer2.apply(lambda x: name_match(x['name1'], x['name2']), axis=1)
    df_layer2["Second_Layer_Pass"] = df_layer2["Second_Layer_Score"] >= threshold
    df_layer3 = df_layer2[~df_layer2["Second_Layer_Pass"]].copy()

    # Layer 3: Initials Matching Without Preprocessing
    df_layer3["Third_Layer_Score"] = df_layer3.apply(lambda x: check_initial_similarity(x['name1'], x['name2']), axis=1)
    df_layer3["Third_Layer_Pass"] = df_layer3["Third_Layer_Score"] >= threshold
    df_layer4 = df_layer3[~df_layer3["Third_Layer_Pass"]].copy()

    # Layer 4: Model and Phonetic Matching
    df_layer4["Fourth_Layer_Score"] = df_layer4.apply(lambda x: name_match(x['name1'], x['name2']), axis=1)
    df_layer4["Fourth_Layer_Pass"] = df_layer4["Fourth_Layer_Score"] >= threshold
    


    # Save layers and results
    df_combined = pd.concat([
        df[df["First_Layer_Pass"]],
        df_layer2[df_layer2["Second_Layer_Pass"]],
        df_layer3[df_layer3["Third_Layer_Pass"]],
        df_layer4[df_layer4["Fourth_Layer_Pass"]],

    ])
    df_combined.to_csv("File.csv",index=False)
    # Updating Prediction status based on layer passes
    df["Prediction"] = False
    df.loc[df["First_Layer_Pass"], "Prediction"] = True
    df.loc[df_layer2.index[df_layer2["Second_Layer_Pass"]], "Prediction"] = True
    df.loc[df_layer3.index[df_layer3["Third_Layer_Pass"]], "Prediction"] = True
    df.loc[df_layer4.index[df_layer4["Fourth_Layer_Pass"]], "Prediction"] = True



    
    # df["Layer0_Pass"] = df.get("Layer0_Pass", None)
    df["First_Layer_Score"] = df.get("First_Layer_Score", None)
    df["First_Layer_Pass"] = df.get("First_Layer_Pass", None)
    df["Second_Layer_Score"] = df_layer2.get("Second_Layer_Score", None)
    df["Second_Layer_Pass"] = df_layer2.get("Second_Layer_Pass", None)
    df["Third_Layer_Score"] = df_layer3.get("Third_Layer_Score", None)
    df["Third_Layer_Pass"] = df_layer3.get("Third_Layer_Pass", None)
    df["Fourth_Layer_Score"] = df_layer4.get("Fourth_Layer_Score", None)
    df["Fourth_Layer_Pass"] = df_layer4.get("Fourth_Layer_Pass", None)
    
   


    # Save the main DataFrame to CSV
    df.to_csv('Api_fun_result.csv')

    # Filter data for labels = 0 and Prediction = True
    filtered_data = df[(df['labels'] == 0) & (df['Prediction'] == True)]

    # Initialize columns for new Prediction and indicator
    df['Updated_Prediction'] = df['Prediction']  # Default to current Prediction
    df['Prediction_Updated'] = False  # Default indicator to False

    # Iterate through the main DataFrame
    for index, row in df.iterrows():
        if index in filtered_data.index:
            # Apply custom logic for rows in filtered_data
            name1 = row['name1']
            name2 = row['name2']
            fuzzy_SS = calculate_fuzzy_similarity_processed(name1, name2)

            # Set fuzzy flag based on similarity threshold
            fuzzy_flag = 1 if fuzzy_SS >= 0.80 else 0
            if fuzzy_flag == 1:
                fuzzy_flag = check_keywords_in_names(name1, name2)

            prediction_value = True if fuzzy_flag == 1 else False

            # Update the Prediction column and set the indicator to True
            df.at[index, 'Updated_Prediction'] = prediction_value
            df.at[index, 'Prediction_Updated'] = True
        else:
            # Leave Updated_Prediction as is and Prediction_Updated as False
            df.at[index, 'Updated_Prediction'] = row['Prediction']

    # Save the updated DataFrame to CSV
    df.to_csv('Filtered_Api_fun_result_with_updates.csv', index=False)

    # Performance metrics using the updated predictions
    y_true = df['labels']
    y_pred = df['Updated_Prediction']
    # y_scores = df['First_Layer_Score']  # Use First_Layer_Score for ROC AUC score
    y_scores = df['Updated_Prediction'].astype(float) 

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1_score": f1_score(y_true, y_pred),
        "roc_auc_score": roc_auc_score(y_true, y_scores),
        "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),
        "classification_report": classification_report(y_true, y_pred, output_dict=True)
    }

    # Correlation matrix (only numeric columns)
    correlation_matrix = df.select_dtypes(include=[np.number]).corr()

    print("Updated DataFrame with new predictions and indicator column:")
    print(df.head())

#    new code ending
    
    
#     data = df[(df['labels'] == 0) & (df['Prediction'] == True)]

#     # Performance metrics
#     y_true = df['labels']
#     y_pred = df['Prediction']
#     y_scores = df["First_Layer_Score"]  # Use the scores for AUC

#     metrics = {
#         "accuracy": accuracy_score(y_true, y_pred),
#         "precision": precision_score(y_true, y_pred),
#         "recall": recall_score(y_true, y_pred),
#         "f1_score": f1_score(y_true, y_pred),
#         "roc_auc_score": roc_auc_score(y_true, y_scores),
#         "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),
#         "classification_report": classification_report(y_true, y_pred, output_dict=True)
#     }

#     # Correlation matrix (only numeric columns)
#     correlation_matrix = df.select_dtypes(include=[np.number]).corr()

    return metrics, df, df_combined, correlation_matrix

# Example usage
if __name__ == "__main__":
    # metrics, full_data, combined_data, correlation_matrix = process_name_matching("2_lakh_NM_Manually.csv", num_rows=10)
    metrics, full_data, combined_data, correlation_matrix = process_name_matching("20k_balaned_data.csv")

    # Print the confusion matrix
    cm = metrics["confusion_matrix"]
    cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
    print("Confusion Matrix:")
    print(cm_df)

    # Print the correlation matrix
    print("\nCorrelation Matrix:")
    print(correlation_matrix)

    # Print other metrics
    print("\nMetrics:")
    print(f"Accuracy: {metrics['accuracy']}")
    print(f"Precision: {metrics['precision']}")
    print(f"Recall: {metrics['recall']}")
    print(f"F1 Score: {metrics['f1_score']}")
    print(f"AUC: {metrics['roc_auc_score']}")


Updated DataFrame with new predictions and indicator column:
         id merchant_type                 created_at     status docType  \
0  47566417   UNORGANIZED  2024-01-01 12:53:41+00:00  ACTIVATED  PAN_NO   
1  48465215   UNORGANIZED  2024-02-14 14:22:48+00:00  ACTIVATED  PAN_NO   
2  48036361   UNORGANIZED  2024-01-29 16:40:52+00:00  ACTIVATED  PAN_NO   
3  46063565   UNORGANIZED  2023-10-06 19:39:01+00:00  ACTIVATED  PAN_NO   
4  45539504           DIY  2023-09-12 19:00:36+00:00  ACTIVATED  PAN_NO   

               name                     pan_createdAt pan_status  \
0             DAILI         2024-01-01 07:36:53+00:00   APPROVED   
1  UMESH KUMAR NAIK  2024-02-14 09:00:02.613000+00:00   APPROVED   
2      KSHAMA GUPTA  2024-01-29 11:18:32.509000+00:00   APPROVED   
3      AFJAL ANSARI         2023-10-06 14:20:17+00:00   APPROVED   
4         AMIN ALAM         2023-09-12 13:38:57+00:00   APPROVED   

          beneficiary_name bank_status  ... First_Layer_Pass  Prediction  \
0  

In [2]:
def check_keywords_in_names(name1, name2):
    """Check for presence of keywords in either or both names."""
    found_in_name1 = any(keyword in name1.lower() for keyword in KEYWORDS)
    found_in_name2 = any(keyword in name2.lower() for keyword in KEYWORDS)
    
    if found_in_name1 and found_in_name2:
        return 1  # Flag remains 1 if keyword found in both names
    elif found_in_name1 or found_in_name2:
        return 0  # Flag set to 0 if keyword found in either name
    return 1

## last layer

In [3]:
import pandas as pd
import re
from fuzzywuzzy import fuzz  # Add this import for fuzzy matching


In [29]:
df_data_fail = pd.read_csv("Api_fun_result.csv")
data = df_data_fail[(df_data_fail['labels'] == 0) & (df_data_fail['Prediction'] == True)]



In [30]:
data.count()


Unnamed: 0            3670
id                    3670
merchant_type         3670
created_at            3670
status                3670
docType               3670
name                  3670
pan_createdAt         3670
pan_status            3670
beneficiary_name      3670
bank_status           3670
account_number        3670
szFlag                   0
hvFlag                 220
name1                 3670
name2                 3670
dsScore               3628
flFlag                3670
dsFlag                3628
szScore                  0
flScore               3670
SCORE                 3670
labels                3670
First_Layer_Score     3670
First_Layer_Pass      3670
Prediction            3670
Second_Layer_Score    3661
Second_Layer_Pass     3661
Third_Layer_Score      506
Third_Layer_Pass       506
Fourth_Layer_Score       0
Fourth_Layer_Pass        0
dtype: int64

In [3]:

KEYWORDS = [
    "traders", "enterprise", "garments", "collection", "food", "clothes",'fashion', 
    "glass", "fittings", "digital", "kirana", "medical", "agency", 
    "security", "systems", "badges", "hospitality", "jewellers", 
    "ready-made", "store", "hospital", "restaurant", "auto", "center", 
    "dairy", "home", "products", "services", "furniture", "hardware", 
    "pharmacy", "stationery", "treatments", "nutrition", "wellness", 
    "sweets", "resort", "kitchen", "clothing", "fashion", "market", 
    "poultry", "seeds", "pesticides", "sales", "cafe", "clinic", 
    "supermart", "distributors", "automobiles", "electricity", 
    "electronics", "general", "provision", "fertilizers", "agriculture", 
    "beverages", "textiles", "plumbing", "supplies", "handicrafts", 
    "construction", "medical", "bakery", "tissue", "cleaning", 
    "appliances", "homecare", "kitchenware", "decor", "glass and fittings",
    "interiors", "shopping", "crafts", "tools", "wholesale", 
    "retail", "outlet", "merchants", "trade", "distribution", 
    "solutions", "innovation", "consultancy", "services", "equipment", 
    "manufacturing", "exports", "imports", "packaging", "network", 
    "consultants", "transport", "moving", "storage", "logistics", 
    "construction", "real estate", "brokerage", "management", 
    "finance", "investment", "funding", "support", "technology", 
    "software", "applications", "digital marketing", "advertising", 
    "communication", "entertainment", "events", "tourism", "travel", 
    "transportation", "automotive", "services", "supply chain", 
    "fashion", "cosmetics", "beauty", "spa", "wellness", "glass and fitting",
    "personal care", "gifts", "custom", "specialty", "craftsmanship", 
    "fashions", "motors", "enterprises", "garment", "cloth centre", "mart", 
    "foods", "silk and readymade", "wool centre", "jewellery", "mill", 
    "farms", "farm", "electrical", "egg centre", "centre", 
    "vegetable and fruits", "vegetables", "fruits", "pvt", "pvt ltd", 
    "limited", "solutions", "energies", "photo", "studio", "works", 
    "associates", "medico", "agencies", "diagnosis", "cool drinks", 
    "drinks", "care", "liquor", "automobiles", "materials", "diagnostics", 
    "provision", "trader", "farms", "farm", "stations", "restaurant", 
    "creations", "travels", "hardware", "printers", "graphics", 
    "fertilisers", "house", "studio", "private", "appliances", "steels", 
    "shop", "metals", "international", "jwellers", "corporation", 
    "dresses", "industries", "electricals", "company", "lim", "colddrinks", 
    "electron", "medicines", "llc", "computers", "hotel", "spa", 
    "cosmetics", "telecom", "sarees", "petroleums", "bhandar",'store','stores', 
    "surgical", "wines", "constructions", "shoppy", "lab", "builders", 
    "footwear", "wear", "shoe", "repair", "ventures", "paint", "depot",'cake','chinies',
    "tent", "decorators", "communications", "pharmacy", "products",'textile','CERAMIC','Pharmaceuticals',
]


SPECIAL_CHAR_DOT_REGEX = r"[.]"
SPECIAL_CHARS_REGEX = r"[-+.^:,_/\s]+" 
SALUTATION_REGEX = r"^(shree|shri|miss|smt|mrs|mr|ms|dr|master|hon|sir|madam|prof|capt|major|rev|fr|br)\s*"
PARENT_SPOUSE_NAME_REGEX = r"(?:\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of)\s*)"
COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX = r"\b(mohammad|mohammed|muhamed|mohd|mohamed|mohamad|muhamad|muhammad|muhammed|muhammet|mohamud|mohummad|mohummed|mouhamed|muhamaad|mohammod|mouhamad|mo|md|mahmood|mahmud|ahmad|ahmed|hameed|hamid|hammed|mahd|mahmod|mohd|mouhammed|mohamad|muhmood|mohhammed|muhmamed|mohmed|mohmat|muhmat|mu|m|shaikh|mo)\b"
LAST_NAMES_AGARWAL_VARIANTS_REGEX = r"\b(aggarwal|agrawal|agarwal|aggrawal|agarwalla|agarwal)\b"


# Preprocessing functions
def convert_to_lower(name):
    return name.lower()

def remove_extra_spaces(name):
    return re.sub(r'\s+', ' ', name).strip()

def remove_common_muslim_variations(text):
    return re.sub(COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_agarwal_variants(text):
    return re.sub(LAST_NAMES_AGARWAL_VARIANTS_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_stop_words(text):
    stop_words = ['devi', 'dei', 'debi', 'kumar', 'kumaar', 'kumari', 'kumaari', 'kmr', 'kumr', 'bhai', 'bhau', 'bai', 'ben', 'singh', 'kaur', 'Md', 'Mohd', 'Mohammad', 'Mohamad']
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


def check_keywords_in_names(name1, name2):
    """Check for presence of keywords in either or both names."""
    found_in_name1 = any(keyword in name1.lower() for keyword in KEYWORDS)
    found_in_name2 = any(keyword in name2.lower() for keyword in KEYWORDS)
    
    if found_in_name1 and found_in_name2:
        return 1  # Flag remains 1 if keyword found in both names
    elif found_in_name1 or found_in_name2:
        return 0  # Flag set to 0 if keyword found in either name
    return 1

 #  processing only for failed  of layer 3
def preprocess_FuzzyWuzzy(name):
    """Process a name through all the defined normalization steps."""
    name = convert_to_lower(name)
    name = remove_stop_words(name)
    name = remove_common_muslim_variations(name)
    name = remove_agarwal_variants(name)
    return name

def calculate_fuzzy_similarity_processed(name1, name2):
    name1 = preprocess_FuzzyWuzzy(name1)
    name2 = preprocess_FuzzyWuzzy(name2)
    
    fuzzy_ratio = fuzz.ratio(name1, name2) / 100.0
    fuzzy_partial_ratio = fuzz.partial_ratio(name1, name2) / 100.0
    fuzzy_token_sort_ratio = fuzz.token_sort_ratio(name1, name2) / 100.0
    fuzzy_token_set_ratio = fuzz.token_set_ratio(name1, name2) / 100.0

    fuzzy_similarity = (fuzzy_ratio + fuzzy_partial_ratio + fuzzy_token_sort_ratio + fuzzy_token_set_ratio) / 4.0
    return fuzzy_similarity



In [40]:
import pandas as pd

results = []
mismatched_data = []

for index, row in data.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    fuzzy_SS = calculate_fuzzy_similarity_processed(name1, name2)

    fuzzy_flag = 1 if fuzzy_SS >= 0.80 else 0
    if fuzzy_flag == 1:
        fuzzy_flag = check_keywords_in_names(name1, name2)

    prediction_value = True if fuzzy_flag == 1 else False

    # Assuming these columns exist in your `data1` DataFrame; adjust if necessary
    result_row = {
        'id': row['id'],
        'name1': name1,
        'name2': name2,
        'dsScore': row['dsScore'],
        'flScore': row['flScore'],
        'SCORE': row['SCORE'],
        'labels': row['labels'],
        'fuzzy_SS': fuzzy_SS,  
        'fuzzy_flag': fuzzy_flag,
        'First_Layer_Score': row['First_Layer_Score'],
        'First_Layer_Pass': row['First_Layer_Pass'],
        'Prediction': prediction_value,
        'Second_Layer_Score': row['Second_Layer_Score'],
        'Second_Layer_Pass': row['Second_Layer_Pass'],
        'Third_Layer_Score': row['Third_Layer_Score'],
        'Third_Layer_Pass': row['Third_Layer_Pass'],
        'Fourth_Layer_Score': row['Fourth_Layer_Score'],
        'Fourth_Layer_Pass': row['Fourth_Layer_Pass']
    }

    if row['labels'] == fuzzy_flag:
        results.append(result_row)
    else:
        mismatched_data.append(result_row)

results_df1 = pd.DataFrame(results)
mismatched_data1_df = pd.DataFrame(mismatched_data)

# Display mismatched data



In [33]:
# name1='SATYENDRA KUMAR PAL'
# name2= 'SATYENDRA PHARMACY'
# fuzzy_SS = calculate_fuzzy_similarity_processed(name1, name2)
# fuzzy_SS
# if fuzzy_flag == 1:
#         fuzzy_flag = check_keywords_in_names(name1, name2)

In [41]:
mismatched_data1_df.count()

id                    30
name1                 30
name2                 30
dsScore               29
flScore               30
SCORE                 30
labels                30
fuzzy_SS              30
fuzzy_flag            30
First_Layer_Score     30
First_Layer_Pass      30
Prediction            30
Second_Layer_Score    25
Second_Layer_Pass     25
Third_Layer_Score      0
Third_Layer_Pass       0
Fourth_Layer_Score     0
Fourth_Layer_Pass      0
dtype: int64

In [42]:
results_df1

,id,name1,name2,dsScore,flScore,SCORE,labels,fuzzy_SS,fuzzy_flag,First_Layer_Score,First_Layer_Pass,Prediction,Second_Layer_Score,Second_Layer_Pass,Third_Layer_Score,Third_Layer_Pass,Fourth_Layer_Score,Fourth_Layer_Pass
0,48465215,UMESH KUMAR NAIK,SUJAL KUMAR,1.72,0.00,1.72,0,0.2325,0,0.5750,False,False,0.953814,True,NaN,NaN,NaN,NaN
1,48659500,KHILENDRA KANT KHARE,KHULESH KANT KHARE S O GYAN DAS KARE,73.63,0.00,73.63,0,0.6250,0,0.6250,False,False,1.026215,True,NaN,NaN,NaN,NaN
2,49415422,KUSHAL MARJIT,KUSHAL GLASS AND FITTINGS,23.22,0.50,23.22,0,0.5250,0,0.5175,False,False,1.021844,True,NaN,NaN,NaN,NaN
3,47694926,GARRE LAKSHMI HARIKA,DIVVELA LAKSHMI HARIKA,0.44,0.00,0.44,0,0.7850,0,0.7850,False,False,1.310948,True,NaN,NaN,NaN,NaN
4,49690637,AMRIT LAL GAUR,THE MODERN MAN,29.00,0.00,29.00,0,0.2925,0,0.2925,False,False,0.809096,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,49952832,AVINASH SRIVASTAVA,SADDA CHINESE ADDA,44.00,0.00,44.00,0,0.4175,0,0.4175,False,False,0.787139,False,1.0,True,NaN,NaN
3636,47951499,KAMAL KUMAR AGARWALA,Ronak Raghuwanshi,38.00,0.00,38.00,0,0.4000,0,0.2750,False,False,0.817186,True,NaN,NaN,NaN,NaN
3637,50771868,SAEED AHMAD,M/S SONA TRADERS,35.00,0.00,35.00,0,0.3350,0,0.3825,False,False,1.045242,True,NaN,NaN,NaN,NaN
3638,47210669,SADANANDAM DESHINI,SHIVA WINES KAMALAPU,26.00,0.00,26.00,0,0.2650,0,0.2650,False,False,0.829727,True,NaN,NaN,NaN,NaN


In [44]:
results_df1.to_csv('fuzzy_fail.csv',index=False)

In [45]:
mismatched_data1_df.to_csv("fuzz_pass.csv",index=False)

In [46]:
mismatched_data1_df

,id,name1,name2,dsScore,flScore,SCORE,labels,fuzzy_SS,fuzzy_flag,First_Layer_Score,First_Layer_Pass,Prediction,Second_Layer_Score,Second_Layer_Pass,Third_Layer_Score,Third_Layer_Pass,Fourth_Layer_Score,Fourth_Layer_Pass
0,47745945,GULASHAN DHAKAD,GULASHAN KUMAR DHAKER,59.58,0.50,59.58,0,0.8700,1,0.7375,False,True,1.364099,True,NaN,NaN,NaN,NaN
1,48249550,M IMRAN KHAN,P IMRAN KHAN,9.91,0.00,9.91,0,0.9550,1,0.9100,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,45531859,SAVITRI DEVI,SAVITRI KUSHWAH,0.37,0.50,0.37,0,0.8200,1,0.6425,False,True,1.323362,True,NaN,NaN,NaN,NaN
3,49546728,Manoj Ramu Sangesaria,MANOJ RAMU VAGRI,58.36,0.00,58.36,0,0.8100,1,0.5225,False,True,1.450239,True,NaN,NaN,NaN,NaN
4,46480124,MOHD BASIM,MOBASIM,75.47,0.50,75.47,0,0.8725,1,0.6775,False,True,1.347481,True,NaN,NaN,NaN,NaN
5,47229736,BADE SIRISHA BHAVANI,Mrs Batchu Sirisha Bhavani,58.23,0.00,58.23,0,0.8050,1,0.5275,False,True,1.369884,True,NaN,NaN,NaN,NaN
6,49205921,KUMKUM DEVI,KUMKUM TRADING,69.98,0.50,69.98,0,0.8000,1,0.6625,False,True,1.267688,True,NaN,NaN,NaN,NaN
7,46740454,MAYA TEX,MAYA DEVI,0.63,0.50,0.63,0,0.8350,1,0.6600,False,True,1.061949,True,NaN,NaN,NaN,NaN
8,48285564,MAMTA DEVI,MAMTA KUMARI,0.40,0.50,0.40,0,1.0000,1,0.6375,False,True,1.010639,True,NaN,NaN,NaN,NaN
9,45517473,MARYAM HAJI MOHAMMED UBEDULLAH KHAN,MARIYAM BEE MOHAMMAD UBEDULLA KHAN,58.70,0.00,58.70,0,0.8200,1,0.8350,True,True,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data['Prediction'].value_counts()

Prediction
True    3670
Name: count, dtype: int64

In [8]:
import re
import pandas as pd

def check_initials_match(name1, name2):
    initials1 = ''.join([word[0] for word in name1.split() if word])
    initials2 = ''.join([word[0] for word in name2.split() if word])
    return initials1.lower() == initials2.lower()

# Function to check if any of the keywords are present in either name
def check_keyword_present(name1, name2, keywords):
    pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    return bool(pattern.search(name1)) or bool(pattern.search(name2))

# Define keywords list
keywords = [
    "traders", "enterprise", "garments", "collection", "food", "clothes",'fashion', 
    "glass", "fittings", "digital", "kirana", "medical", "agency", 
    "security", "systems", "badges", "hospitality", "jewellers", 
    "ready-made", "store", "hospital", "restaurant", "auto", "center", 
    "dairy", "home", "products", "services", "furniture", "hardware", 
    "pharmacy", "stationery", "treatments", "nutrition", "wellness", 
    "sweets", "resort", "kitchen", "clothing", "fashion", "market", 
    "poultry", "seeds", "pesticides", "sales", "cafe", "clinic", 
    "supermart", "distributors", "automobiles", "electricity", 
    "electronics", "general", "provision", "fertilizers", "agriculture", 
    "beverages", "textiles", "plumbing", "supplies", "handicrafts", 
    "construction", "medical", "bakery", "tissue", "cleaning", 
    "appliances", "homecare", "kitchenware", "decor", "glass and fittings",
    "interiors", "shopping", "crafts", "tools", "wholesale", 
    "retail", "outlet", "merchants", "trade", "distribution", 
    "solutions", "innovation", "consultancy", "services", "equipment", 
    "manufacturing", "exports", "imports", "packaging", "network", 
    "consultants", "transport", "moving", "storage", "logistics", 
    "construction", "real estate", "brokerage", "management", 
    "finance", "investment", "funding", "support", "technology", 
    "software", "applications", "digital marketing", "advertising", 
    "communication", "entertainment", "events", "tourism", "travel", 
    "transportation", "automotive", "services", "supply chain", 
    "fashion", "cosmetics", "beauty", "spa", "wellness", "glass and fitting",
    "personal care", "gifts", "custom", "specialty", "craftsmanship", 
    "fashions", "motors", "enterprises", "garment", "cloth centre", "mart", 
    "foods", "silk and readymade", "wool centre", "jewellery", "mill", 
    "farms", "farm", "electrical", "egg centre", "centre", 
    "vegetable and fruits", "vegetables", "fruits", "pvt", "pvt ltd", 
    "limited", "solutions", "energies", "photo", "studio", "works", 
    "associates", "medico", "agencies", "diagnosis", "cool drinks", 
    "drinks", "care", "liquor", "automobiles", "materials", "diagnostics", 
    "provision", "trader", "farms", "farm", "stations", "restaurant", 
    "creations", "travels", "hardware", "printers", "graphics", 
    "fertilisers", "house", "studio", "private", "appliances", "steels", 
    "shop", "metals", "international", "jwellers", "corporation", 
    "dresses", "industries", "electricals", "company", "lim", "colddrinks", 
    "electron", "medicines", "llc", "computers", "hotel", "spa", 
    "cosmetics", "telecom", "sarees", "petroleums", "bhandar",'store','stores', 
    "surgical", "wines", "constructions", "shoppy", "lab", "builders", 
    "footwear", "wear", "shoe", "repair", "ventures", "paint", "depot",'cake','chinies',
    "tent", "decorators", "communications", "pharmacy", "products",'textile','CERAMIC','Pharmaceuticals',
]


#-------------------
# Function to apply initials and keywords checks
def apply_initials_and_keyword_checks(name1, name2, keywords):
    if check_initials_match(name1, name2):
        if check_keyword_present(name1, name2, keywords):
            return False
        else:
            return True
    else:
        return False


data["Prediction"] = data.apply(
    lambda x: apply_initials_and_keyword_checks(x['name1'], x['name2'], keywords) if x["Prediction"] else x["Prediction"],
    axis=1
)



/var/tmp/ipykernel_393976/2110763975.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Prediction"] = data.apply(


In [9]:
data['Prediction'].value_counts()

Prediction
False    3614
True       56
Name: count, dtype: int64

In [10]:
data.to_csv('Pass.csv',index=False)

In [2]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import (
#     accuracy_score, 
#     confusion_matrix, 
#     precision_score, 
#     recall_score, 
#     f1_score, 
#     roc_auc_score, 
#     classification_report
# )

# def process_name_matching(file_path):
# # def process_name_matching(file_path, num_rows=10):

#     # Load data
#     df = pd.read_csv(file_path)
#     # df = pd.read_csv(file_path, nrows=num_rows)
#     threshold = 0.80

#     # Layer 1: Fuzzy Matching Without Preprocessing
#     df["First_Layer_Score"] = df.apply(lambda x: calculate_fuzzy_similarity(x['name1'], x['name2']), axis=1)
#     df["First_Layer_Pass"] = df["First_Layer_Score"] >= threshold
#     df_layer2 = df[~df["First_Layer_Pass"]].copy()

#     # Layer 2: Preprocessed Fuzzy Matching
#     df_layer2["Second_Layer_Score"] = df_layer2.apply(lambda x: calculate_fuzzy_similarity_processed(x['name1'], x['name2']), axis=1)
#     df_layer2["Second_Layer_Pass"] = df_layer2["Second_Layer_Score"] >= threshold
#     df_layer3 = df_layer2[~df_layer2["Second_Layer_Pass"]].copy()

#     # Layer 3: Initials Matching Without Preprocessing
#     df_layer3["Third_Layer_Score"] = df_layer3.apply(lambda x: check_initial_similarity(x['name1'], x['name2']), axis=1)
#     df_layer3["Third_Layer_Pass"] = df_layer3["Third_Layer_Score"] >= threshold
#     df_layer4 = df_layer3[~df_layer3["Third_Layer_Pass"]].copy()

#     # Layer 4: Model and Phonetic Matching
#     df_layer4["Fourth_Layer_Score"] = df_layer4.apply(lambda x: name_match(x['name1'], x['name2']), axis=1)
#     df_layer4["Fourth_Layer_Pass"] = df_layer4["Fourth_Layer_Score"] >= threshold
    

#     # Save layers and results
#     df_combined = pd.concat([
#         df[df["First_Layer_Pass"]],
#         df_layer2[df_layer2["Second_Layer_Pass"]],
#         df_layer3[df_layer3["Third_Layer_Pass"]],
#         df_layer4[df_layer4["Fourth_Layer_Pass"]],

#     ])
#     df_combined.to_csv("File.csv",index=False)
#     # Updating Prediction status based on layer passes
#     df["Prediction"] = False
#     df.loc[df["First_Layer_Pass"], "Prediction"] = True
#     df.loc[df_layer2.index[df_layer2["Second_Layer_Pass"]], "Prediction"] = True
#     df.loc[df_layer3.index[df_layer3["Third_Layer_Pass"]], "Prediction"] = True
#     df.loc[df_layer4.index[df_layer4["Fourth_Layer_Pass"]], "Prediction"] = True



    
#     # df["Layer0_Pass"] = df.get("Layer0_Pass", None)
#     df["First_Layer_Score"] = df.get("First_Layer_Score", None)
#     df["First_Layer_Pass"] = df.get("First_Layer_Pass", None)
#     df["Second_Layer_Score"] = df_layer2.get("Second_Layer_Score", None)
#     df["Second_Layer_Pass"] = df_layer2.get("Second_Layer_Pass", None)
#     df["Third_Layer_Score"] = df_layer3.get("Third_Layer_Score", None)
#     df["Third_Layer_Pass"] = df_layer3.get("Third_Layer_Pass", None)
#     df["Fourth_Layer_Score"] = df_layer4.get("Fourth_Layer_Score", None)
#     df["Fourth_Layer_Pass"] = df_layer4.get("Fourth_Layer_Pass", None)
    


#     df.to_csv('Api_fun_result.csv')


#     # Performance metrics
#     y_true = df['labels']
#     y_pred = df['Prediction']
#     y_scores = df["First_Layer_Score"]  # Use the scores for AUC

#     metrics = {
#         "accuracy": accuracy_score(y_true, y_pred),
#         "precision": precision_score(y_true, y_pred),
#         "recall": recall_score(y_true, y_pred),
#         "f1_score": f1_score(y_true, y_pred),
#         "roc_auc_score": roc_auc_score(y_true, y_scores),
#         "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),
#         "classification_report": classification_report(y_true, y_pred, output_dict=True)
#     }

#     # Correlation matrix (only numeric columns)
#     correlation_matrix = df.select_dtypes(include=[np.number]).corr()

#     return metrics, df, df_combined, correlation_matrix

# # Example usage
# if __name__ == "__main__":
#     # metrics, full_data, combined_data, correlation_matrix = process_name_matching("2_lakh_NM_Manually.csv", num_rows=10)
#     metrics, full_data, combined_data, correlation_matrix = process_name_matching("20k_balaned_data.csv")

#     # Print the confusion matrix
#     cm = metrics["confusion_matrix"]
#     cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
#     print("Confusion Matrix:")
#     print(cm_df)

#     # Print the correlation matrix
#     print("\nCorrelation Matrix:")
#     print(correlation_matrix)

#     # Print other metrics
#     print("\nMetrics:")
#     print(f"Accuracy: {metrics['accuracy']}")
#     print(f"Precision: {metrics['precision']}")
#     print(f"Recall: {metrics['recall']}")
#     print(f"F1 Score: {metrics['f1_score']}")
#     print(f"AUC: {metrics['roc_auc_score']}")


Keyword match found: False


In [ ]:
# import pandas as pd

# def process_name_matching_two(name1, name2):
#     threshold = 0.80

#     # Layer 1: Fuzzy Matching Without Preprocessing
#     first_layer_score = calculate_fuzzy_similarity(name1, name2)
#     if first_layer_score >= threshold:
#         return first_layer_score

#     # Layer 2: Preprocessed Fuzzy Matching
#     second_layer_score = calculate_fuzzy_similarity_processed(name1, name2)
#     if second_layer_score >= threshold:
#         return second_layer_score

#     # Layer 3: Initials Matching Without Preprocessing
#     third_layer_score = check_initial_similarity(name1, name2)
#     if third_layer_score >= threshold:
#         return third_layer_score

#     # Layer 4: Model and Phonetic Matching
#     fourth_layer_score = name_match(name1, name2)
#     if fourth_layer_score >= threshold:
#         return fourth_layer_score

#     # If none of the layers meet the threshold, return the last computed score or None
#     return fourth_layer_score

# # Example usage
# if __name__ == "__main__":
#     name1 = "Manmeet Singh "
#     name2 = "Singh Manmeet"
#     similarity_score = process_name_matching_two(name1, name2)
#     print(f"\nSimilarity score between '{name1}' and '{name2}': {similarity_score}")


In [36]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics import (
#     accuracy_score, 
#     confusion_matrix, 
#     precision_score, 
#     recall_score, 
#     f1_score, 
#     roc_auc_score, 
#     classification_report
# )

# def process_name_matching(file_path):
# # def process_name_matching(file_path, num_rows=10):

#     # Load data
#     df = pd.read_csv(file_path)
#     # df = pd.read_csv(file_path, nrows=num_rows)
#     threshold = 0.80

#     # Layer 1: Fuzzy Matching Without Preprocessing
#     df["First_Layer_Score"] = df.apply(lambda x: calculate_fuzzy_similarity(x['name1'], x['name2']), axis=1)
#     df["First_Layer_Pass"] = df["First_Layer_Score"] >= threshold
#     df_layer2 = df[~df["First_Layer_Pass"]].copy()

#     # Layer 2: Preprocessed Fuzzy Matching
#     df_layer2["Second_Layer_Score"] = df_layer2.apply(lambda x: calculate_fuzzy_similarity_processed(x['name1'], x['name2']), axis=1)
#     df_layer2["Second_Layer_Pass"] = df_layer2["Second_Layer_Score"] >= threshold
#     df_layer3 = df_layer2[~df_layer2["Second_Layer_Pass"]].copy()

#     # Layer 3: Initials Matching Without Preprocessing
#     df_layer3["Third_Layer_Score"] = df_layer3.apply(lambda x: check_initial_similarity(x['name1'], x['name2']), axis=1)
#     df_layer3["Third_Layer_Pass"] = df_layer3["Third_Layer_Score"] >= threshold
#     df_layer4 = df_layer3[~df_layer3["Third_Layer_Pass"]].copy()

#     # Layer 4: Model and Phonetic Matching
#     df_layer4["Fourth_Layer_Score"] = df_layer4.apply(lambda x: name_match(x['name1'], x['name2']), axis=1)
#     df_layer4["Fourth_Layer_Pass"] = df_layer4["Fourth_Layer_Score"] >= threshold

#     # Save layers and results
#     df_combined = pd.concat([
#         df[df["First_Layer_Pass"]],
#         df_layer2[df_layer2["Second_Layer_Pass"]],
#         df_layer3[df_layer3["Third_Layer_Pass"]],
#         df_layer4[df_layer4["Fourth_Layer_Pass"]]
#     ])
#     df_combined.to_csv("File.csv",index=False)
#     # Updating Prediction status based on layer passes
#     df["Prediction"] = False
#     df.loc[df["First_Layer_Pass"], "Prediction"] = True
#     df.loc[df_layer2.index[df_layer2["Second_Layer_Pass"]], "Prediction"] = True
#     df.loc[df_layer3.index[df_layer3["Third_Layer_Pass"]], "Prediction"] = True
#     df.loc[df_layer4.index[df_layer4["Fourth_Layer_Pass"]], "Prediction"] = True

#     # Performance metrics
#     y_true = df['labels']
#     y_pred = df['Prediction']
#     y_scores = df["First_Layer_Score"]  # Use the scores for AUC

#     metrics = {
#         "accuracy": accuracy_score(y_true, y_pred),
#         "precision": precision_score(y_true, y_pred),
#         "recall": recall_score(y_true, y_pred),
#         "f1_score": f1_score(y_true, y_pred),
#         "roc_auc_score": roc_auc_score(y_true, y_scores),
#         "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),
#         "classification_report": classification_report(y_true, y_pred, output_dict=True)
#     }

#     # Correlation matrix (only numeric columns)
#     correlation_matrix = df.select_dtypes(include=[np.number]).corr()

#     return metrics, df, df_combined, correlation_matrix

# # Example usage
# if __name__ == "__main__":
#     # metrics, full_data, combined_data, correlation_matrix = process_name_matching("2_lakh_NM_Manually.csv", num_rows=10)
#     metrics, full_data, combined_data, correlation_matrix = process_name_matching("20k_balaned_data.csv")

#     # Print the confusion matrix
#     cm = metrics["confusion_matrix"]
#     cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
#     print("Confusion Matrix:")
#     print(cm_df)

#     # Print the correlation matrix
#     print("\nCorrelation Matrix:")
#     print(correlation_matrix)

#     # Print other metrics
#     print("\nMetrics:")
#     print(f"Accuracy: {metrics['accuracy']}")
#     print(f"Precision: {metrics['precision']}")
#     print(f"Recall: {metrics['recall']}")
#     print(f"F1 Score: {metrics['f1_score']}")
#     print(f"AUC: {metrics['roc_auc_score']}")


Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0         6661         3670
Actual 1            1        10170

Correlation Matrix:
                         id  szFlag   dsScore  szScore   flScore     SCORE  \
id                 1.000000     NaN -0.027826      NaN  0.019066  0.020089   
szFlag                  NaN     NaN       NaN      NaN       NaN       NaN   
dsScore           -0.027826     NaN  1.000000      NaN  0.100611  0.998056   
szScore                 NaN     NaN       NaN      NaN       NaN       NaN   
flScore            0.019066     NaN  0.100611      NaN  1.000000  0.893569   
SCORE              0.020089     NaN  0.998056      NaN  0.893569  1.000000   
labels             0.020322     NaN  0.516676      NaN  0.930848  0.935229   
First_Layer_Score  0.006455     NaN  0.449603      NaN  0.862994  0.850939   

                     labels  First_Layer_Score  
id                 0.020322           0.006455  
szFlag                  NaN                NaN  
dsScor

In [70]:
import re


# Layer 0 - Check for Keyword Match
def check_keyword_match(name1, name2, keywords):
    pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    name1_contains_keyword = bool(pattern.search(name1))
    name2_contains_keyword = bool(pattern.search(name2))
    return name1_contains_keyword and name2_contains_keyword

# Provided keywords list
keywords = [
    "traders", "enterprise", "garments", "collection", "food", "clothes", 
    "glass", "fittings", "digital", "kirana", "medical", "agency", 
    "security", "systems", "badges", "hospitality", "jewellers", 
    "ready-made", "store", "hospital", "restaurant", "auto", "center", 
    "dairy", "home", "products", "services", "furniture", "hardware", 
    "pharmacy", "stationery", "treatments", "nutrition", "wellness", 
    "sweets", "resort", "kitchen", "clothing", "fashion", "market", 
    "poultry", "seeds", "pesticides", "sales", "cafe", "clinic", 
    "supermart", "distributors", "automobiles", "electricity", 
    "electronics", "general", "provision", "fertilizers", "agriculture", 
    "beverages", "textiles", "plumbing", "supplies", "handicrafts", 
    "construction", "medical", "bakery", "tissue", "cleaning", 
    "appliances", "homecare", "kitchenware", "decor", "glass and fittings",
    "interiors", "shopping", "crafts", "tools", "wholesale", 
    "retail", "outlet", "merchants", "trade", "distribution", 
    "solutions", "innovation", "consultancy", "services", "equipment", 
    "manufacturing", "exports", "imports", "packaging", "network", 
    "consultants", "transport", "moving", "storage", "logistics", 
    "construction", "real estate", "brokerage", "management", 
    "finance", "investment", "funding", "support", "technology", 
    "software", "applications", "digital marketing", "advertising", 
    "communication", "entertainment", "events", "tourism", "travel", 
    "transportation", "automotive", "services", "supply chain", 
    "fashion", "cosmetics", "beauty", "spa", "wellness", "glass and fitting",
    "personal care", "gifts", "custom", "specialty", "craftsmanship", 
    "fashions", "motors", "enterprises", "garment", "cloth centre", "mart", 
    "foods", "silk and readymade", "wool centre", "jewellery", "mill", 
    "farms", "farm", "electrical", "egg centre", "centre", 
    "vegetable and fruits", "vegetables", "fruits", "pvt", "pvt ltd", 
    "limited", "solutions", "energies", "photo", "studio", "works", 
    "associates", "medico", "agencies", "diagnosis", "cool drinks", 
    "drinks", "care", "liquor", "automobiles", "materials", "diagnostics", 
    "provision", "trader", "farms", "farm", "stations", "restaurant", 
    "creations", "travels", "hardware", "printers", "graphics", 
    "fertilisers", "house", "studio", "private", "appliances", "steels", 
    "shop", "metals", "international", "jwellers", "corporation", 
    "dresses", "industries", "electricals", "company", "lim", "colddrinks", 
    "electron", "medicines", "llc", "computers", "hotel", "spa", 
    "cosmetics", "telecom", "sarees", "petroleums", "bhandar", 
    "surgical", "wines", "constructions", "shoppy", "lab", "builders", 
    "footwear", "wear", "shoe", "repair", "ventures", "paint", "depot", 
    "tent", "decorators", "communications", "pharmacy", "products"
]

# # Layer 0 - Check for Keyword Match
# def check_keyword_match(name1, name2, keywords):
#     pattern = re.compile('|'.join(keywords), re.IGNORECASE)
#     name1_contains_keyword = bool(pattern.search(name1))
#     name2_contains_keyword = bool(pattern.search(name2))
#     return name1_contains_keyword and name2_contains_keyword


# Test examples
name1 = "Royal Traders"
name2 = "Green Leaf Pharmaceuticals"

# Check if any keywords are found in either name
result = check_keyword_match(name1, name2, keywords)
print("Keyword match found:", result)


Keyword match found: False


In [82]:
import re

def check_keyword_match(name1, name2, keywords):
    # Create a regex pattern to match any of the keywords, case-insensitive
    pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    
    # Check if name1 and name2 contain any of the keywords
    name1_contains_keyword = bool(pattern.search(name1))
    name2_contains_keyword = bool(pattern.search(name2))

    # Return True if both name1 and name2 contain at least one keyword, else False
    return int(name1_contains_keyword and name2_contains_keyword)

# Example usage
keywords = [
    "traders", "enterprise", "garments", "collection", "food", "clothes", 
    "glass", "fittings", "digital", "kirana", "medical", "agency", 
    "security", "systems", "badges", "hospitality", "jewellers", 
    "ready-made", "store", "hospital", "restaurant", "auto", "center", 
    "dairy", "home", "products", "services", "furniture", "hardware", 
    "pharmacy", "stationery", "treatments", "nutrition", "wellness", 
    "sweets", "resort", "kitchen", "clothing", "fashion", "market", 
    "poultry", "seeds", "pesticides", "sales", "cafe", "clinic", 
    "supermart", "distributors", "automobiles", "electricity", 
    "electronics", "general", "provision", "fertilizers", "agriculture", 
    "beverages", "textiles", "plumbing", "supplies", "handicrafts", 
    "construction", "medical", "bakery", "tissue", "cleaning", 
    "appliances", "homecare", "food", "kitchenware", "decor", 
    "interiors", "shopping", "crafts", "tools", "wholesale", 
    "retail", "outlet", "merchants", "trade", "distribution", 
    "solutions", "innovation", "consultancy", "services", "equipment", 
    "manufacturing", "exports", "imports", "packaging", "network", 
    "consultants", "transport", "moving", "storage", "logistics", 
    "construction", "real estate", "brokerage", "management", 
    "finance", "investment", "funding", "support", "technology", 
    "software", "applications", "digital marketing", "advertising", 
    "communication", "entertainment", "events", "tourism", "travel", 
    "transportation", "automotive", "services", "supply chain", 
    "fashion", "cosmetics", "beauty", "spa", "wellness", 
    "personal care", "gifts", "custom", "specialty", "craftsmanship"
]

name1 = "MOHAMMED MOHSIN MOHAMMED SALIM SHAIKH"
name2 = "TAJ ENTERPRISES"
result = check_keyword_match(name1, name2, keywords)
print("Prediction:", result)  # This will print "Prediction: True" if both contain keywords, else "False"


Prediction: 0


In [ ]:
1 min bssss